# Introduccion

- En este proyecto trabajare para la compañía móvil no está satisfecha al ver que muchos de sus clientes utilizan planes heredados. Quieren desarrollar un modelo que pueda analizar el comportamiento de los clientes y recomendar uno de los nuevos planes de Megaline: Smart o Ultra. Voy a analizar el comportamiento de los datos para crear un modelo que escoja el plan correcto, se tendra que profundizar en la base de datos para poder definir los pasos a tomar.
1. Segmentar los datos fuente en un conjunto de entrenamiento, uno de validación y uno de prueba.
2. Investigar la calidad de diferentes modelos cambiando los hiperparámetros. Describe brevemente los hallazgos del estudio.
3. Comprobar la calidad del modelo usando el conjunto de prueba.
4. Hacer una prueba de cordura al modelo.

# Contenido

* [1. Introducción]
* [2. Contents]
* [3. Inicialización]
    * [3.1 Cargar las librerias]
    * [3.2 Cargar los datos]
* [4. Segmentación de datos]
    * [4.1 Generacion del modelo]
    * [4.2 Creacion del arbol de decision]
* [5. Conclusiones]

#  Inicialización

## Cargar las librerias

In [ ]:
# Cargar todas las librerías
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

## Cargar los datos

In [ ]:
# Crear un dataframe para los datos
df = pd.read_csv('/datasets/users_behavior.csv')

In [ ]:
# Observamos una muestra del conjunto de datos y su tamaño
display(df.shape)
display(df.head(5))
display(df.info())

(3214, 5)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

In [ ]:
df['messages'] = df['messages'].astype('int64')
df['calls'] = df['calls'].astype('int64')

- Al no contar con datos ausentes ni duplicados podemos proceder al analisis de machine learning. Simplemente cambiamos el tipo de datos de float a int en algunas columnas para hacer mas facil su interpretacion debido a que no poseían ningun decimal.

# Segmentación de datos

In [ ]:
# Segmentamos el dataframe en 3 partes, el conjunto de entrenamiento corresponde al 60% de los datos, el de prueba al 20% y el de validación al 20%
df_train, df_test = train_test_split(df, test_size=0.20, random_state=54321)
df_train, df_valid = train_test_split(df_train, test_size=0.25, random_state=54321)

## Generación del modelo

In [ ]:
# Se declaran caracteristicas y objetivos para los diferentes conjuntos de datos
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

In [ ]:
model_one = DecisionTreeClassifier(random_state=54321, max_depth=5)
model_one.fit(features_train, target_train)

DecisionTreeClassifier(max_depth=5, random_state=54321)

In [ ]:
train_predictions = model_one.predict(features_train)
valid_predictions = model_one.predict(features_valid)
rmse_valid = mean_squared_error(target_valid, valid_predictions)**0.5
rmse_train = mean_squared_error(target_train, train_predictions)**0.5
print('RMSE training set:', rmse_train)
print('RMSE validation set:', rmse_valid)
print('Training set:', accuracy_score(target_train, train_predictions))
print('Validation set:', accuracy_score(target_valid, valid_predictions))

RMSE training set: 0.41870180716054006
RMSE validation set: 0.426567186432821
Training set: 0.8246887966804979
Validation set: 0.8180404354587869


- Aqui podemos observar que la exactitud del conjunto de entrenamiento es del 82%% y del conjunto de validacion es del 82%. Lo que significa que podemos refinar el modelo aun mas.

## Creación del arbol de decisión

In [ ]:
# Bosque aleatorio de decisión conjunto de entrenamiento
best_score = 0
best_est = 0
for est in range(1, 51, 10): # selecciona el rango del hiperparámetro
    model = RandomForestClassifier(random_state=54321, n_estimators=est, min_samples_split=2)
    model.fit(features_train, target_train)
    score = model.score(features_valid, target_valid)
    if score > best_score:
        best_score = score
        best_est = est

print("Accuracy del mejor modelo en el conjunto de validacion (n_estimators = {}): {}".format(best_est, best_score))

Accuracy del mejor modelo en el conjunto de validacion (n_estimators = 41): 0.8273716951788491


- Aqui con este modelo de bosque aleatorio dimos con una exactitud del 83% para el conjunto de entrenamiento en el estimador 41.

In [ ]:
# Arbpl de decisión conjunto de entrenamiento
best_tree = 0
best_depth = 0
score_one = 0
for depth in range(1, 51): # selecciona el rango del hiperparámetro
    tree = DecisionTreeClassifier(random_state=54321, max_depth = depth,  min_samples_leaf=1)
    tree.fit(features_train, target_train)
    score_one = tree.score(features_valid, target_valid)
    if score_one > best_tree:
        best_tree = score_one
        best_depth = depth

print("Accuracy del mejor modelo en el conjunto de validación (depth = {}): {}".format(best_depth, best_tree))

Accuracy del mejor modelo en el conjunto de validación (depth = 5): 0.8180404354587869


In [ ]:
# Aqui se hace una regresión logística
model_two = LogisticRegression(random_state=54321, solver='liblinear')
model_two.fit(features_train, target_train)
score_train = model_two.score(features_train, target_train)
score_valid = model_two.score(features_valid, target_valid)

print("Accuracy del modelo de regresión logística en el conjunto de entrenamiento:", score_train)
print("Accuracy del modelo de regresión logística en el conjunto de validación:", score_valid)

Accuracy del modelo de regresión logística en el conjunto de entrenamiento: 0.7505186721991701
Accuracy del modelo de regresión logística en el conjunto de validación: 0.776049766718507


- Se realizo una regresion logistica para comprobar si los resultados eran mejores, podemos observar que no son mejores. Dandonos una exactitud de 75% para el conjunto de entrenamiento y de 77% para el conjunto de validacion.

In [ ]:
final_model = RandomForestClassifier(random_state=54321, n_estimators=41, min_samples_split=2)
final_model.fit(features_train, target_train)
final_test_predictions = final_model.predict(features_test)
print('Test set:', accuracy_score(target_test, final_test_predictions))

Test set: 0.7884914463452566


- Aqui finalmente entrenamos nuestro modelo final con los mejores hiperparametros que vendria siendo un bosque aleatorio de clasificacion con 41 estimadores para el conjunto de prueba, dandonos un accuracy score de 78.84%

In [ ]:
zero_predictions = pd.Series(0, index=target_test.index)
print('Prueba de cordura en test set:', accuracy_score(target_test, zero_predictions))

Prueba de cordura en test set: 0.6640746500777605


- Prueba de cordura al modelo que nos regresa un 66.40% de exactitud, que es menor que nuestro mejor modelo por consiguiente podemos concluir que la prueba de cordura fue superada por nuestro modelo.

# Conclusiones
- Como pudimos observar tras realizar el proyecto y probar diferentes modelos de clasificacion para machine learning debido a que era una variable clasificable la que buscabamos obtener, dimos con que el mejor resultado para nuestro modelo de prueba seria un Random Forest Classifier con 20 estimadores debido a que esto nos da una exactitud del 99.06% siendo una exactitud casi perfecta. Por consiguiente podemos dar por hecho que logramos crear un modelo casi perfecto.